In [1]:
import theano

In [2]:
import numpy as np
from time import time
import theano.tensor as T
import cPickle
from theano.tensor.nnet import softmax
#dtype = theano.config.dtype
dtype='float32'

In [3]:
i2w=["<pad>","un","deux","trois","quatre","cinq","six","sept","huit","neuf"]

In [4]:
index_fixe=[0,3]

In [5]:
i2f=[0,0.4,0.5,0.6,0.7,0.8,0.88,0.95,0.98,1]

In [6]:
i2f[5:]

[0.8, 0.88, 0.95, 0.98, 1]

In [7]:
vocab_size = len(i2w)
dim = 5

In [8]:
W = np.vstack((np.zeros(dim),0.001*np.random.randn(vocab_size-1, dim))).astype(dtype)
V = np.vstack((np.zeros(dim),0.001*np.random.randn(vocab_size-1, dim))).astype(dtype)
#0.001*np.random.randn(vocab_size, dim).astype(dtype)

In [9]:
W

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  3.67685920e-04,  -8.18518165e-04,  -5.51570905e-04,
         -1.92546682e-03,   1.05576590e-03],
       [  1.34588347e-03,   9.56688717e-04,   5.80201915e-04,
         -1.03196164e-03,  -5.70894801e-04],
       [ -1.38826872e-05,   4.03675251e-04,   5.35573869e-04,
          5.08184312e-04,  -1.56225334e-03],
       [  6.72577822e-04,   6.94144575e-04,  -2.75114289e-04,
          1.55758287e-04,   7.61797011e-04],
       [ -1.28774752e-03,   2.64709495e-04,   3.02798318e-04,
          9.55128489e-05,  -2.17208668e-04],
       [  1.20640034e-03,  -2.24424433e-03,   1.00623246e-03,
         -3.25804955e-04,  -3.85297375e-04],
       [  9.70565248e-04,   7.27406470e-04,  -4.23940801e-04,
         -8.22612259e-04,  -1.10004032e-04],
       [ -2.76636361e-04,  -2.41060276e-04,  -1.49319856e-03,
          1.69037061e-03,   9.59230005e-04],
       [  1.07968540e-03,   1.3653517

In [10]:
W = theano.shared(W, name='W')
V = theano.shared(V, name='V')

In [11]:
indexes = T.imatrix('X')

batch_mask = 1-T.eq(indexes, T.zeros_like(indexes))
mask = batch_mask[1:-1]
maskp = batch_mask[:-2]
maskn = batch_mask[2:]
X_indexes = indexes[1:-1]
Yp_indexes = indexes[:-2]
Yn_indexes = indexes[2:]

In [132]:
batch_len,sent_len=X_indexes.shape

In [14]:
indexes1=np.array([[1,7,8,0],[2,4,0,0],[2,4,0,0],[1,8,9,4]],dtype='int32')

In [15]:
indexX=X_indexes.flatten()

W_index=W[indexX]

In [16]:
X=W_index.reshape((batch_len,sent_len,-1))

In [18]:
Xs = T.sum(X, axis=1)
Xs.eval({indexes:indexes1})

array([[ 0.00201846,  0.00165083,  0.00030509, -0.0008762 ,  0.0001909 ],
       [ 0.00201846,  0.00165083,  0.00030509, -0.0008762 ,  0.0001909 ]], dtype=float32)

In [23]:
neg_len=10

In [24]:
from random import random
from bisect import bisect


def weighted_choice():
    x = random()
    i = bisect(i2f, x)
    return i

In [25]:
neg=[weighted_choice() for i in range(neg_len)]

In [26]:
I = [Yp_indexes, Yn_indexes]
I = T.concatenate(I, axis=1)
M = [maskp, maskn]
M=T.concatenate(M, axis=1)

In [27]:
max_len=I.shape[1]

pos=I.flatten()

In [28]:
index=T.concatenate([pos,neg])

In [114]:
index.eval({indexes:indexes1})

array([1, 7, 8, 0, 2, 4, 0, 0, 2, 4, 0, 0, 1, 8, 9, 4, 7, 1, 1, 1, 6, 1, 1,
       1, 5, 1])

In [29]:
V_index=V[index]

In [30]:
batch_len.eval({indexes:indexes1})

array(2)

In [31]:
V_index.eval({indexes:indexes1})[0:5]

array([[-0.00036857, -0.00073604,  0.00017893,  0.00055846, -0.00102126],
       [-0.00066097,  0.0005526 , -0.00016924,  0.0004105 ,  0.00098169],
       [ 0.00092678,  0.00116544, -0.00107586,  0.00037091,  0.00031154],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.00151739,  0.00021228,  0.00211028,  0.00071778, -0.00114865]], dtype=float32)

In [32]:
activations = T.dot(Xs, V_index.T)

In [33]:
activations.eval({indexes:indexes1})

array([[ -2.58871228e-06,  -6.45793875e-07,   3.20087065e-06,
          0.00000000e+00,  -2.91673041e-06,  -3.07839855e-06,
          0.00000000e+00,   0.00000000e+00,  -2.91673041e-06,
         -3.07839855e-06,   0.00000000e+00,   0.00000000e+00,
         -2.58871228e-06,   3.20087065e-06,  -1.93550886e-06,
         -3.07839855e-06,  -6.45793875e-07,  -2.58871228e-06,
         -2.58871228e-06,  -2.58871228e-06,   2.15239879e-06,
         -2.58871228e-06,  -2.58871228e-06,  -2.58871228e-06,
         -1.11156749e-06,  -2.58871228e-06],
       [ -2.58871228e-06,  -6.45793875e-07,   3.20087065e-06,
          0.00000000e+00,  -2.91673041e-06,  -3.07839855e-06,
          0.00000000e+00,   0.00000000e+00,  -2.91673041e-06,
         -3.07839855e-06,   0.00000000e+00,   0.00000000e+00,
         -2.58871228e-06,   3.20087065e-06,  -1.93550886e-06,
         -3.07839855e-06,  -6.45793875e-07,  -2.58871228e-06,
         -2.58871228e-06,  -2.58871228e-06,   2.15239879e-06,
         -2.58871228e-06,

In [34]:
chelou_line=T.arange(I.shape[0]).reshape((-1,1))


In [35]:
pos_columns=T.arange(batch_len*max_len).reshape((batch_len,-1))

In [36]:
neg_columns=(T.arange(batch_len*max_len,(batch_len*max_len+neg_len)))

In [37]:
shaped_neg_columns=(neg_columns.reshape((1,-1)))

In [38]:
shaped_neg_columns.eval({indexes:indexes1})

array([[16, 17, 18, 19, 20, 21, 22, 23, 24, 25]])

In [39]:
pos_columns.eval({indexes:indexes1},)

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15]])

In [40]:
rep_neg_columns=T.repeat(shaped_neg_columns,batch_len,axis=0)

In [41]:
rep_neg_columns.eval({indexes:indexes1},)

array([[16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
       [16, 17, 18, 19, 20, 21, 22, 23, 24, 25]])

In [42]:
chelou_column=T.concatenate((pos_columns,rep_neg_columns),axis=1)

chelou_column.eval({indexes:indexes1})

array([[ 0,  1,  2,  3,  4,  5,  6,  7, 16, 17, 18, 19, 20, 21, 22, 23, 24,
        25],
       [ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
        25]])

In [43]:
acti_posneg=activations[chelou_line,chelou_column]

In [44]:
acti_posneg.eval({indexes:indexes1})

array([[ -2.58871228e-06,  -6.45793875e-07,   3.20087065e-06,
          0.00000000e+00,  -2.91673041e-06,  -3.07839855e-06,
          0.00000000e+00,   0.00000000e+00,  -6.45793875e-07,
         -2.58871228e-06,  -2.58871228e-06,  -2.58871228e-06,
          2.15239879e-06,  -2.58871228e-06,  -2.58871228e-06,
         -2.58871228e-06,  -1.11156749e-06,  -2.58871228e-06],
       [ -2.91673041e-06,  -3.07839855e-06,   0.00000000e+00,
          0.00000000e+00,  -2.58871228e-06,   3.20087065e-06,
         -1.93550886e-06,  -3.07839855e-06,  -6.45793875e-07,
         -2.58871228e-06,  -2.58871228e-06,  -2.58871228e-06,
          2.15239879e-06,  -2.58871228e-06,  -2.58871228e-06,
         -2.58871228e-06,  -1.11156749e-06,  -2.58871228e-06]], dtype=float32)

In [45]:
mask_neg_zero=T.zeros_like(rep_neg_columns)

In [46]:
mask_neg=(1-mask_neg_zero)

In [47]:
mask_full = T.concatenate((M,mask_neg),axis=1)

In [48]:
mask_zero=T.concatenate((M,mask_neg_zero),axis=1)

In [49]:
acti_mask=acti_posneg*mask_full

In [50]:
acti_mask.eval({indexes:indexes1})

array([[ -2.58871228e-06,  -6.45793875e-07,   3.20087065e-06,
          0.00000000e+00,  -2.91673041e-06,  -3.07839855e-06,
          0.00000000e+00,   0.00000000e+00,  -6.45793875e-07,
         -2.58871228e-06,  -2.58871228e-06,  -2.58871228e-06,
          2.15239879e-06,  -2.58871228e-06,  -2.58871228e-06,
         -2.58871228e-06,  -1.11156749e-06,  -2.58871228e-06],
       [ -2.91673041e-06,  -3.07839855e-06,   0.00000000e+00,
          0.00000000e+00,  -2.58871228e-06,   3.20087065e-06,
         -1.93550886e-06,  -3.07839855e-06,  -6.45793875e-07,
         -2.58871228e-06,  -2.58871228e-06,  -2.58871228e-06,
          2.15239879e-06,  -2.58871228e-06,  -2.58871228e-06,
         -2.58871228e-06,  -1.11156749e-06,  -2.58871228e-06]])

In [51]:
prediction = softmax(acti_mask)

prediction.eval({indexes:indexes1})

array([[ 0.05555549,  0.05555559,  0.05555581,  0.05555563,  0.05555547,
         0.05555546,  0.05555563,  0.05555563,  0.05555559,  0.05555549,
         0.05555549,  0.05555549,  0.05555575,  0.05555549,  0.05555549,
         0.05555549,  0.05555557,  0.05555549],
       [ 0.05555548,  0.05555547,  0.05555564,  0.05555564,  0.0555555 ,
         0.05555582,  0.05555553,  0.05555547,  0.05555561,  0.0555555 ,
         0.0555555 ,  0.0555555 ,  0.05555576,  0.0555555 ,  0.0555555 ,
         0.0555555 ,  0.05555558,  0.0555555 ]])

In [52]:
logpred=T.log(prediction)*mask_zero

In [53]:
prediction.eval({indexes:indexes1})

array([[ 0.05555549,  0.05555559,  0.05555581,  0.05555563,  0.05555547,
         0.05555546,  0.05555563,  0.05555563,  0.05555559,  0.05555549,
         0.05555549,  0.05555549,  0.05555575,  0.05555549,  0.05555549,
         0.05555549,  0.05555557,  0.05555549],
       [ 0.05555548,  0.05555547,  0.05555564,  0.05555564,  0.0555555 ,
         0.05555582,  0.05555553,  0.05555547,  0.05555561,  0.0555555 ,
         0.0555555 ,  0.0555555 ,  0.05555576,  0.0555555 ,  0.0555555 ,
         0.0555555 ,  0.05555558,  0.0555555 ]])

In [54]:
cost = T.mean(T.sum(-T.log(prediction)*mask_zero, axis=1))

In [55]:
cost.eval({indexes:indexes1})

array(15.897044894855453)

In [55]:
lr = T.scalar('lr', dtype=dtype)

In [58]:
params = [W_index,V_index]
#not W but subtensor of W

In [59]:
params_name=["W_index","V_index"]

In [60]:
grads = T.grad(cost, params)

In [73]:
grads

[Reshape{2}.0, DimShuffle{1,0}.0]

In [74]:
W_index.eval({indexes:indexes1})

array([[ 0.00134579,  0.00095667,  0.00058025, -0.00103203, -0.00057065],
       [ 0.00067248,  0.00069412, -0.00027506,  0.00015569,  0.00076204],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.00134579,  0.00095667,  0.00058025, -0.00103203, -0.00057065],
       [ 0.00067248,  0.00069412, -0.00027506,  0.00015569,  0.00076204],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]], dtype=float32)

In [72]:
grads[0].eval({indexes:indexes1})

array([[ 0.0004001 , -0.00060944, -0.00016033, -0.00017079, -0.00098464],
       [ 0.0004001 , -0.00060944, -0.00016033, -0.00017079, -0.00098464],
       [ 0.0004001 , -0.00060944, -0.00016033, -0.00017079, -0.00098464],
       [ 0.0004001 , -0.00060944, -0.00016033, -0.00017079, -0.00098464],
       [ 0.00046487,  0.00010898, -0.00026042,  0.00033778, -0.00122052],
       [ 0.00046487,  0.00010898, -0.00026042,  0.00033778, -0.00122052],
       [ 0.00046487,  0.00010898, -0.00026042,  0.00033778, -0.00122052],
       [ 0.00046487,  0.00010898, -0.00026042,  0.00033778, -0.00122052]], dtype=float32)

In [78]:
lr=1

In [79]:
updates = [(p, p-lr*g) for p,g in zip(params, grads)]



In [116]:
updates = []
sub_update={}

for p,g,n in zip(params, grads,params_name):
    sub_update[n]=-lr*g

In [95]:
set([1,2])

{1, 2}

In [110]:
test=np.array([1,2,3,4])

In [111]:
test[[1,2,2]]=[0,1,2]

In [112]:
test

array([1, 0, 2, 4])

In [115]:
T.inc_subtensor?

In [125]:
updateV=T.inc_subtensor(V[index],sub_update["V_index"] )
updateV=T.set_subtensor(updateV[index_fixe],V[index_fixe])

In [119]:
updateW=T.inc_subtensor(W[X_flatten],sub_update["W_index"])
updateW=T.set_subtensor(updateW[index_fixe],W[index_fixe])

In [127]:
index.eval({indexes:indexes1})==1

array([ True, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False,  True,
        True,  True, False,  True,  True,  True, False,  True], dtype=bool)

In [131]:
np.sum(grads[1].eval({indexes:indexes1})[index.eval({indexes:indexes1})==1],axis=0)

array([ 0.00291528,  0.00238447,  0.00044083, -0.00126582,  0.00027645], dtype=float32)

In [88]:
X_flatten.eval({indexes:indexes1})

array([2, 4, 0, 0, 2, 4, 0, 0], dtype=int32)

In [120]:
grads[0].eval({indexes:indexes1})

array([[ 0.0004001 , -0.00060944, -0.00016033, -0.00017079, -0.00098464],
       [ 0.0004001 , -0.00060944, -0.00016033, -0.00017079, -0.00098464],
       [ 0.0004001 , -0.00060944, -0.00016033, -0.00017079, -0.00098464],
       [ 0.0004001 , -0.00060944, -0.00016033, -0.00017079, -0.00098464],
       [ 0.00046487,  0.00010898, -0.00026042,  0.00033778, -0.00122052],
       [ 0.00046487,  0.00010898, -0.00026042,  0.00033778, -0.00122052],
       [ 0.00046487,  0.00010898, -0.00026042,  0.00033778, -0.00122052],
       [ 0.00046487,  0.00010898, -0.00026042,  0.00033778, -0.00122052]], dtype=float32)

In [122]:
-0.00017079+0.00033778


0.00016698999999999997

In [124]:
updateV.eval({indexes:indexes1})- V.get_value()

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [ -2.91528110e-03,  -2.38446938e-03,  -4.40831733e-04,
          1.26581988e-03,  -2.76452163e-04],
       [  1.40158087e-03,   1.14638230e-03,   2.11938983e-04,
         -6.08568720e-04,   1.32910092e-04],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  2.07433989e-03,   1.69664598e-03,   3.13669530e-04,
         -9.00681829e-04,   1.96706969e-04],
       [ -6.16695383e-04,  -5.04408032e-04,  -9.32530675e-05,
          2.67770141e-04,  -5.84804220e-05],
       [ -6.16697420e-04,  -5.04409720e-04,  -9.32533294e-05,
          2.67770956e-04,  -5.84806257e-05],
       [  1.12125650e-04,   9.17099533e-05,   1.69550040e-05,
         -4.86851495e-05,   1.06327934e-05],
       [  1.40157726e-03,   1.14637928e-03,   2.11938284e-04,
         -6.08567032e-04,   1.32909772e-04],
       [  6.72758557e-04,   5.5026327

In [84]:
updates.append((V,updateV))
updates.append((W,updateW))

In [67]:
fun_cost=theano.function(inputs = [indexes], outputs = cost,updates=updates,allow_input_downcast=True,mode='DebugMode')

In [68]:
fun_cost(indexes1)

array(15.897044894855668)

In [69]:
fun_cost(indexes1)

array(15.897044893037961)

In [70]:
W.get_value()

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  3.67685920e-04,  -8.18518165e-04,  -5.51570905e-04,
         -1.92546682e-03,   1.05576590e-03],
       [  1.34579046e-03,   9.56666830e-04,   5.80253953e-04,
         -1.03202916e-03,  -5.70650678e-04],
       [ -1.38826872e-05,   4.03675251e-04,   5.35573869e-04,
          5.08184312e-04,  -1.56225334e-03],
       [  6.72484806e-04,   6.94122689e-04,  -2.75062193e-04,
          1.55690752e-04,   7.62041134e-04],
       [ -1.28774752e-03,   2.64709495e-04,   3.02798318e-04,
          9.55128489e-05,  -2.17208668e-04],
       [  1.20640034e-03,  -2.24424433e-03,   1.00623246e-03,
         -3.25804955e-04,  -3.85297375e-04],
       [  9.70565248e-04,   7.27406470e-04,  -4.23940801e-04,
         -8.22612259e-04,  -1.10004032e-04],
       [ -2.76636361e-04,  -2.41060276e-04,  -1.49319856e-03,
          1.69037061e-03,   9.59230005e-04],
       [  1.07968540e-03,   1.3653517

In [66]:
V.get_value()

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [ -1.17721513e-03,   1.37171010e-03,   4.95452899e-04,
         -2.25335592e-04,  -3.79721663e-04],
       [  8.76754057e-04,  -1.00329053e-05,   5.86552080e-04,
          1.77010486e-04,  -2.66592228e-03],
       [ -1.00104534e-03,   1.13159430e-03,   3.40168102e-04,
         -5.45030460e-04,  -9.77109070e-04],
       [  8.98642640e-04,   3.40753119e-04,  -1.39003649e-04,
         -1.35730347e-03,  -1.22595171e-03],
       [  1.48651819e-03,  -1.19569944e-04,  -8.63898313e-05,
         -2.14076065e-03,  -2.30405573e-03],
       [ -1.29415351e-03,   8.74533318e-04,  -3.04370676e-03,
         -4.19470336e-04,   6.09797426e-05],
       [ -9.03261942e-04,  -2.16961955e-04,   1.07093761e-03,
         -1.96898705e-03,   2.56974273e-03],
       [ -1.71902985e-03,  -4.98983427e-04,  -3.27271613e-04,
          2.98244599e-03,   9.83668840e-04],
       [  4.17386735e-04,  -3.2588272

In [67]:
V.get_value()

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [ -1.17721513e-03,   1.37171010e-03,   4.95452899e-04,
         -2.25335592e-04,  -3.79721663e-04],
       [  8.76754057e-04,  -1.00329053e-05,   5.86552080e-04,
          1.77010486e-04,  -2.66592228e-03],
       [ -1.00104534e-03,   1.13159430e-03,   3.40168102e-04,
         -5.45030460e-04,  -9.77109070e-04],
       [  8.98642640e-04,   3.40753119e-04,  -1.39003649e-04,
         -1.35730347e-03,  -1.22595171e-03],
       [  1.48651819e-03,  -1.19569944e-04,  -8.63898313e-05,
         -2.14076065e-03,  -2.30405573e-03],
       [ -1.29415351e-03,   8.74533318e-04,  -3.04370676e-03,
         -4.19470336e-04,   6.09797426e-05],
       [ -9.03261942e-04,  -2.16961955e-04,   1.07093761e-03,
         -1.96898705e-03,   2.56974273e-03],
       [ -1.71902985e-03,  -4.98983427e-04,  -3.27271613e-04,
          2.98244599e-03,   9.83668840e-04],
       [  4.17386735e-04,  -3.2588272

In [139]:
import cPickle

In [141]:
with open(r"chineseModel", "rb") as input_file:
    print input_file
    e = cPickle.load(input_file)

<open file 'chineseModel', mode 'rb' at 0x1082db8a0>


EOFError: 

In [161]:
input_file=open(r"chineseModel", "r")

In [162]:
e = cPickle.load(input_file)

In [163]:
e

[True,
 W,
 V,
 {u'##1156402': 41,
  u'##12453': 44,
  u'##186953': 39,
  u'##207961': 19,
  u'##673661': 42,
  u'##79782': 29,
  u'##8078': 10,
  u'#11129#': 6,
  u'#181319#': 17,
  u'#22622#': 14,
  '<pad>': 0,
  u'\u4e00\u7a2e': 24,
  u'\u4e00\u9580': 5,
  u'\u4e09\u4e16\u7d00': 20,
  u'\u4e4b\u7236': 12,
  u'\u4ee5\u53ca': 22,
  u'\u4f5c\u54c1': 36,
  u'\u4f7f\u7528': 13,
  u'\u5229\u7528': 11,
  u'\u53d8\u5316': 15,
  u'\u5bb6': 27,
  u'\u5c0d\u7269': 9,
  u'\u5c6c\u65bc': 46,
  u'\u5e0c\u81d8': 48,
  u'\u5e7e\u4f55': 25,
  u'\u5f62\u5f0f': 23,
  u'\u6570\u5b66': 1,
  u'\u6578': 45,
  u'\u6578\u5b78': 2,
  u'\u6578\u91cf': 21,
  u'\u67d0\u79cd': 47,
  u'\u6982\u5ff5': 43,
  u'\u6b50\u5e7e\u91cc\u5f97': 18,
  u'\u73fe': 49,
  u'\u7522\u751f': 40,
  u'\u7531\u8a08': 37,
  u'\u756b\u70ba': 30,
  u'\u7814\u7a76': 32,
  u'\u79d1\u5b78': 33,
  u'\u7a7a\u95f4': 3,
  u'\u7b26\u53f7': 8,
  u'\u7b49': 26,
  u'\u7ed3\u6784': 4,
  u'\u897f\u5143\u524d': 28,
  u'\u89c0\u5bdf': 16,
  u'\u89d2\u

In [165]:
e[3]['##1156402']

41